In [1]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

from src.utils import read_config, OracleAgent

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb


from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


from llama_index.core.schema import Document

c:\Users\kira.chou\AppData\Local\miniconda3\envs\kira-project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### 查詢根目錄
import os
os.getcwd()

'd:\\11_coding\\Git\\Kira-Project'

In [3]:
info_path ='.env/info.json'
info = read_config(info_path)
##info

In [4]:
info['DW_conn_info']

{'user': 'YFYDW',
 'pw': '!(Yfy123456)',
 'host': '10.25.66.64',
 'port': '1521',
 'service_name': 'biprod'}

In [5]:
Oracle_Agent = OracleAgent(info['DW_conn_info'])

In [6]:
query =query = """
select rowid, dashboard_name, report_name, page_name, pic_name, llm as description, llm_question as question_candidates
from bi_dashboard
"""

df_dashboard = Oracle_Agent.read_table(query=query)
df_dashboard.head(2)

,rowid,dashboard_name,report_name,page_name,pic_name,description,question_candidates
0,AAi0soABZAAAACLAAC,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...,這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\...
1,AAi0soABZAAAACLAAF,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...,這張圖片顯示了華紙的營運績效分析月報，可以回答以下問題：\n\n1. **華紙的總成本是多少...


In [7]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

In [15]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("DashboardQA")

In [16]:
### list comprehension

def df_to_documents(df):
   return [Document(text=row['description'], metadata={col: row[col] for col in df.columns if col != 'description'}) for _, row in df.iterrows()]

In [17]:

df_dashboard.head(2)

,rowid,dashboard_name,report_name,page_name,pic_name,description,question_candidates
0,AAi0soABZAAAACLAAC,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...,這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\...
1,AAi0soABZAAAACLAAF,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...,這張圖片顯示了華紙的營運績效分析月報，可以回答以下問題：\n\n1. **華紙的總成本是多少...


In [18]:
documents=df_to_documents(df_dashboard)

In [19]:
# store in vectordb
Settings.chunk_size = 1024
Settings.chunk_overlap = 50
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
   documents=documents,
   storage_context=storage_context,
   embed_model=embeddings
)

In [21]:
# load from disk

db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
   vector_store,
   embed_model=embeddings,
)

In [26]:
# Query Data from the persisted index
query_engine = index.as_retriever(similarity_top_k=3)
response = query_engine.retrieve("""              
哪張表格包含
淨利、稅前淨利、稅後淨利
""")
display(Markdown(f"{response[0]}"))


Node ID: 7341202f-8f9c-406d-84d6-6af38124410e
Text: This table shows the monthly operational performance analysis of
Hua Shi paper making machine profit and loss.  It includes the
following analysis indicators:  **Product:** PM1, PM2, PM7, Total,
Combined total  **Product Types:**  Raw materials, finished products,
subtotals   **Indicators:**    * Sales revenue * Cost of goods sold *
Gross profit...
Score:  0.229


In [25]:
response

[NodeWithScore(node=TextNode(id_='7341202f-8f9c-406d-84d6-6af38124410e', embedding=None, metadata={'rowid': 'AAi0soABZAAAACXAAG', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '紙機別損益', 'pic_name': '紙機別損益-完整畫面.jpg', 'question_candidates': '這張圖片看起來像一張營運績效分析月報表，它呈現了華紙的營運狀況，可以回答以下問題：\n\n1. 這是哪個月的報表？\n2. 報表中顯示了哪些紙機別？\n3. 每個紙機別的銷售額是多少？\n4. 每個紙機別的毛利是多少？\n5. 每個紙機別的毛利率是多少？\n6. 每個紙機別的營運成本是多少？\n7. 每個紙機別的淨利是多少？\n8. 每個紙機別的淨利率是多少？\n9. 不同紙機別的營運表現有什麼差異？\n10. 總體而言，華紙的營運表現如何？\n'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9549ee48-98b2-4223-892a-2ff8a5af2c6d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'rowid': 'AAi0soABZAAAACXAAG', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '紙機別損益', 'pic_name': '紙機別損益-完整畫面.jpg', 'question_candidates': '這張圖片看起來像一張營運績效分析月報表，它呈現了華紙的營運狀況，可以回答以下問題：\n\n1. 這是哪個月的報表？\n2. 報表中顯示了哪些紙機別？\n3. 每個紙機別的銷售額是多少？\n4. 每個紙機別的毛利是多少？\n5. 每個紙機別